In [ ]:
pip install python-dotenv

In [ ]:
pip install openai

In [ ]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())    

pip install --upgrade openai

In [ ]:
client = OpenAI()

In [ ]:
 llm_model = "gpt-3.5-turbo"

In [ ]:
def get_completion(prompt, model=llm_model):  

# for choice in chat_completion.choices:
#     print(choice.message.content)
 
    
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
get_completion("What is 1+1?")


In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
style = """American English \
in a calm, respectful, panicked women tone
"""

In [ ]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)


In [ ]:
response = get_completion(prompt)

In [ ]:
response

In [ ]:
pip install --upgrade langchain

In [ ]:
pip install langchain-community


In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [ ]:
prompt_template

In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
customer_style = """Indian English \
in a calm, respectful and 5 year old tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

In [ ]:
print(customer_messages[0])

In [ ]:
customer_response = chat.invoke(customer_messages)

In [ ]:
print(customer_response.content)

In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [ ]:
service_response = chat.invoke(service_messages)
print(service_response.content)

### Parsers


In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}


In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

In [ ]:
type(response.content)

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

In [ ]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

In [ ]:
response = chat(messages)

In [ ]:
print(response.content)

In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

In [ ]:
output_dict.get('delivery_days')

### Memory

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [ ]:
 llm_model = "gpt-3.5-turbo"
    

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
from langchain.callbacks import StdOutCallbackHandler

# Initialize callback
callback_handler = StdOutCallbackHandler()

In [ ]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True,
    callbacks=[callback_handler]
)

In [ ]:
pip install --upgrade langchain

In [ ]:
conversation.predict(input="Hi, my name is Fuzail")

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="What is my name?")

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

### ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(k=2)

In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [ ]:
memory.load_memory_variables({})

In [ ]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="Hi, my name is Ansari")

In [ ]:
conversation.predict(input="What is 1+1?")

In [ ]:
conversation.predict(input="What is my name?")

### ConversationTokenBufferMemory

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [ ]:
memory.load_memory_variables({})

### ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your development team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the indian resturant with a customer who is driving \
from over an hour away to meet you to understand the how ur work fits in their current proejct need. \
Be sure to bring your laptop to show any demo or resume to customer."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show to customer?")

In [ ]:
memory.load_memory_variables({})

# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
llm_model = "gpt-3.5-turbo"

In [ ]:
pip install pandas

In [ ]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [ ]:
df.head()

## LLMChain


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "Wireless Mouse"
chain.run(product)

## SimpleSequentialChain`

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.run(product)

## SequentialChain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [ ]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [ ]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
review = df.ReviewText[2]
print(review)

In [ ]:
overall_chain(review)

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner for class 7 student. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question for class 7 student.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements for class 7 student.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity for class 7 student. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
destinations

In [ ]:
destinations_str

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
router_prompt

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is Lang Chain?")

In [ ]:
chain.run("What is MetaVerse          ?")

In [ ]:
chain.run("What is Fibanacci Numbers?")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

### LangChain: Q&A over Documents

# TO GET FILE from jupiter notebook 
from IPython.display import FileLink

#Display a link to download the CSV file
file = 'OutdoorClothingCatalog_1000.csv'
FileLink(file)

In [ ]:
pip install --upgrade langchain

In [ ]:
pip install langchain_community

In [ ]:
pip install --upgrade docarray

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
llm_model = "gpt-3.5-turbo"

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI

In [ ]:
file = 'sample_outdoor_clothing_catalog_with_description.csv'
loader = CSVLoader(file_path=file)

In [ ]:
import pandas as pd
from io import StringIO
from langchain.document_loaders import CSVLoader

# Specify the file path
file = 'OutdoorClothingCatalog_1000.csv'

# Read the CSV file with the appropriate encoding
try:
    df = pd.read_csv(file, encoding='ISO-8859-1')  # Adjust the encoding if needed
except UnicodeDecodeError:
    df = pd.read_csv(file, encoding='utf-8')  # Try utf-8 if ISO-8859-1 doesn't work

# Convert the DataFrame back to a CSV string
csv_data = df.to_csv(index=False)

# Use StringIO to simulate a file for CSVLoader
csv_file_like = StringIO(csv_data)

# Load the CSV data into CSVLoader
loader = CSVLoader(file_path=csv_file_like)

# Use loader as needed for further processing
documents = loader.load()

# Print or use the loaded documents
print(documents)


In [ ]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
pip install faiss-cpu

In [ ]:
import pandas as pd
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Step 1: Read the CSV file with correct encoding using pandas
file = 'sample_outdoor_clothing_catalog_with_description.csv'

# Try reading the file with ISO-8859-1 encoding, fallback to UTF-8
try:
    df = pd.read_csv(file, encoding='ISO-8859-1')
except UnicodeDecodeError:
    df = pd.read_csv(file, encoding='utf-8')

# Step 2: Save the corrected CSV file back to disk
corrected_file = 'corrected_outdoor_clothing.csv'
df.to_csv(corrected_file, index=False)

# Step 3: Use CSVLoader to load the corrected CSV file
loader = CSVLoader(file_path=corrected_file)

# Step 4: Create an embedding model (e.g., OpenAIEmbeddings)
embeddings = OpenAIEmbeddings()

# Step 5: Create a vectorstore index using FAISS and the CSVLoader
index_creator = VectorstoreIndexCreator(
    embedding=embeddings,        # Required embedding model
    vectorstore_cls=FAISS        # FAISS is used for the vector store
)

# Step 6: Build the index using the loader
index = index_creator.from_loaders([loader])

# Now you can use the index as needed

print(index)

In [ ]:
import pandas as pd
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Step 4: Create an embedding model (e.g., OpenAIEmbeddings)
embeddings = OpenAIEmbeddings()

# Step 5: Create a vectorstore index using FAISS and the CSVLoader
index_creator = VectorstoreIndexCreator(
    embedding=embeddings,        # Required embedding model
    vectorstore_cls=FAISS        # FAISS is used for the vector store
)

# Step 6: Build the index using the loader
index = index_creator.from_loaders([loader])

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [ ]:
llm_replacement_model = OpenAI(temperature=0, 
                               model='gpt-3.5-turbo-instruct')

response = index.query(query, 
                       llm = llm_replacement_model)

In [ ]:
display(Markdown(response))

## Step By Step

In [ ]:
from langchain.document_loaders import CSVLoader
file = 'sample_outdoor_clothing_catalog_with_description.csv'
loader = CSVLoader(file_path=file)

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

In [ ]:
from langchain.embeddings import OpenAIEmbeddings


In [ ]:
embeddings = OpenAIEmbeddings()
embed = embeddings.embed_query("Hi my name is Fuzail")

In [ ]:
print(len(embed))

In [ ]:
print(embed[:5])

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [ ]:
query = "Please suggest a shirt with sunblocking"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
len(docs)

In [ ]:
docs[0]

In [ ]:
retriever = db.as_retriever()

In [ ]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [ ]:
display(Markdown(response))

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts and summarize each one."

In [ ]:
response = qa_stuff.run(query)

In [ ]:
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

# LangChain: Evaluation

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
    llm_model = "gpt-3.5-turbo"

In [4]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [5]:
from langchain.document_loaders import CSVLoader
file = 'sample_outdoor_clothing_catalog_with_description.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Step 4: Create an embedding model (e.g., OpenAIEmbeddings)
embeddings = OpenAIEmbeddings()

# Step 5: Create a vectorstore index using FAISS and the CSVLoader
index = VectorstoreIndexCreator(
    embedding=embeddings,        # Required embedding model
    vectorstore_cls=FAISS        # FAISS is used for the vector store
).from_loaders([loader])

C:\Users\mohansar2\AppData\Local\Temp\ipykernel_38264\3857346684.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [7]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

C:\Users\mohansar2\AppData\Local\Temp\ipykernel_38264\1284373331.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature = 0.0, model=llm_model)


In [50]:
data[0]


Document(metadata={'source': 'sample_outdoor_clothing_catalog_with_description.csv', 'row': 0}, page_content="S.No: 0\nName: Women's Campside Oxfords\nDescription: Comfortable and stylish, perfect for outdoor activities.")

In [62]:
data[1]

Document(metadata={'source': 'sample_outdoor_clothing_catalog_with_description.csv', 'row': 1}, page_content='S.No: 1\nName: Recycled Waterhog Dog Mat, Chevron Weave\nDescription: Eco-friendly, durable dog mat made from recycled materials.')

In [63]:
data[2]

Document(metadata={'source': 'sample_outdoor_clothing_catalog_with_description.csv', 'row': 2}, page_content="S.No: 2\nName: Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece\nDescription: Cute, comfortable two-piece swimsuit for toddlers, perfect for the beach.")

In [8]:
examples = [
    {
        "query": "is Women's Campside Oxfords suited for outdoor activity?",
        "answer": "Yes"
    },
    {
        "query": "swimsuit for toddlers?",
        "answer": "Coastal Chill Swimsuit"
    }
]

In [9]:
from langchain.evaluation.qa import QAGenerateChain

In [10]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=llm_model))

In [11]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

C:\Users\mohansar2\AppData\Local\anaconda3\Lib\site-packages\langchain\chains\llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [12]:
new_examples

[{'qa_pairs': {'query': 'What is the name and description of the product described in the document?',
   'answer': "The name of the product is Women's Campside Oxfords, and the description states that they are comfortable and stylish, perfect for outdoor activities."}},
 {'qa_pairs': {'query': 'What is the name and description of the first item in the outdoor clothing catalog?',
   'answer': 'The name of the first item is Recycled Waterhog Dog Mat, Chevron Weave. The description states that it is an eco-friendly, durable dog mat made from recycled materials.'}},
 {'qa_pairs': {'query': 'What is the name and description of the swimsuit for toddlers mentioned in the document?',
   'answer': 'The name of the swimsuit for toddlers is "Infant and Toddler Girls\' Coastal Chill Swimsuit, Two-Piece" and the description is "Cute, comfortable two-piece swimsuit for toddlers, perfect for the beach."'}},
 {'qa_pairs': {'query': 'What is the name and description of the product listed as S.No 3 in t

In [69]:
examples += new_examples

In [ ]:
qa.run(examples[0]["query"])

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


In [1]:
predictions = qa.apply(examples)

NameError: name 'qa' is not defined

In [13]:
predictions = qa.apply(examples)

C:\Users\mohansar2\AppData\Local\Temp\ipykernel_38264\1205324748.py:1: LangChainDeprecationWarning: The method `Chain.apply` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~batch` instead.
  predictions = qa.apply(examples)
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.

> Finished chain.


In [14]:
predictions

[{'query': "is Women's Campside Oxfords suited for outdoor activity?",
  'answer': 'Yes',
  'result': "Yes, Women's Campside Oxfords are described as comfortable and stylish, perfect for outdoor activities."},
 {'query': 'swimsuit for toddlers?',
  'answer': 'Coastal Chill Swimsuit',
  'result': 'The swimsuit for toddlers is the "Infant and Toddler Girls\' Coastal Chill Swimsuit, Two-Piece." It is described as cute, comfortable, and perfect for the beach.'}]

In [15]:
from langchain.evaluation.qa import QAEvalChain

In [16]:
llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [17]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [19]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    #print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: is Women's Campside Oxfords suited for outdoor activity?
Real Answer: Yes
Predicted Answer: Yes, Women's Campside Oxfords are described as comfortable and stylish, perfect for outdoor activities.

Example 1:
Question: swimsuit for toddlers?
Real Answer: Coastal Chill Swimsuit
Predicted Answer: The swimsuit for toddlers is the "Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece." It is described as cute, comfortable, and perfect for the beach.



In [ ]:
pre